In [2]:
import numpy as np
import pandas as pd
import pickle

df = pd.read_csv('../data/processed/scores.csv', dtype={'refinement': object})
df.refinement = df.refinement.fillna('')

In [7]:
df = df.groupby(['dataset', 'embedding', 'refinement']).mean()[['score_1', 'score_5', 'score_10']].sort_values('score_10', ascending=False).reset_index()

df['dim'] = 0
for i, row in df.iterrows():
    dataset, embedding, refinement = row.dataset, row.embedding, row.refinement
    fname = f"../data/interim/{dataset.split('.')[0]}_{embedding.replace('/', '-')}{refinement}.pkl"
    with open(fname, "rb") as f:
        embeddings = pickle.load(f)
    embeddings = np.array(embeddings)

    # Calculate effective dimension using SVD
    u, s, vh = np.linalg.svd(embeddings - embeddings.mean(axis=1, keepdims=True), full_matrices=False)
    effective_dim = (s ** 2).sum() / (s[0] ** 2)

    df.loc[i, 'dim'] = embeddings.shape[1]
    df.loc[i, 'effective_dim'] = effective_dim

df.round(3).groupby(['dataset', 'embedding', 'refinement']).first().sort_values('score_10', ascending=False)


/tmp/ipykernel_175460/1381850096.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['dataset', 'embedding', 'refinement']).mean()[['score_1', 'score_5', 'score_10']].sort_values('score_10', ascending=False).reset_index()


score_1  \
dataset                embedding                               refinement            
sfn_2015_subsample.csv sentence-transformers/all-mpnet-base-v2 1580415070    1.183   
                                                               1150734315    1.175   
                                                               1795493411    1.174   
                                                               276397127     1.176   
                                                               1817992910    1.178   
                                                               360739329     1.167   
                                                               508533280     1.169   
                                                                             1.150   
                       sentence-transformers/all-MiniLM-L6-v2                1.152   
                       hkunlp/instructor-xl                                  1.148   
                       hkunlp/instructor-large                               1.133   
                       sentence-transformers/allenai-specter                 1.040   
                       text-embedding-ada-002                                1.026   
                       intfloat/e5-base                                      1.072   
                       sentence-transformers/gtr-t5-large                    0.979   
                       intfloat/e5-large                                     0.302   

                                                                           score_5  \
dataset                embedding                               refinement            
sfn_2015_subsample.csv sentence-transformers/all-mpnet-base-v2 1580415070    1.084   
                                                               1150734315    1.080   
                                                               1795493411    1.083   
                                                               276397127     1.086   
                                                               1817992910    1.082   
                                                               360739329     1.083   
                                                               508533280     1.082   
                                                                             1.027   
                       sentence-transformers/all-MiniLM-L6-v2                0.992   
                       hkunlp/instructor-xl                                  0.996   
                       hkunlp/instructor-large                               0.964   
                       sentence-transformers/allenai-specter                 0.917   
                       text-embedding-ada-002                                0.918   
                       intfloat/e5-base                                      0.905   
                       sentence-transformers/gtr-t5-large                    0.789   
                       intfloat/e5-large                                     0.251   

                                                                           score_10  \
dataset                embedding                               refinement             
sfn_2015_subsample.csv sentence-transformers/all-mpnet-base-v2 1580415070     1.027   
                                                               1150734315     1.024   
                                                               1795493411     1.024   
                                                               276397127      1.024   
                                                               1817992910     1.021   
                                                               360739329      1.018   
                                                               508533280      1.017   
                                                                              0.944   
                       sentence-transformers/all-MiniLM-L6-v2                 0.900   
                    

In [8]:
print(df.round(3).groupby(['dataset', 'embedding', 'refinement']).first().sort_values('score_10', ascending=False).to_markdown())

|                                                                                     |   score_1 |   score_5 |   score_10 |   dim |   effective_dim |
|:------------------------------------------------------------------------------------|----------:|----------:|-----------:|------:|----------------:|
| ('sfn_2015_subsample.csv', 'sentence-transformers/all-mpnet-base-v2', '1580415070') |     1.183 |     1.084 |      1.027 |   512 |           2.477 |
| ('sfn_2015_subsample.csv', 'sentence-transformers/all-mpnet-base-v2', '1150734315') |     1.175 |     1.08  |      1.024 |    64 |           2.496 |
| ('sfn_2015_subsample.csv', 'sentence-transformers/all-mpnet-base-v2', '1795493411') |     1.174 |     1.083 |      1.024 |  1024 |           2.456 |
| ('sfn_2015_subsample.csv', 'sentence-transformers/all-mpnet-base-v2', '276397127')  |     1.176 |     1.086 |      1.024 |   128 |           2.525 |
| ('sfn_2015_subsample.csv', 'sentence-transformers/all-mpnet-base-v2', '1817992910') |     1.

In [3]:
df_hp = pd.read_parquet('../data/processed/hyperparameter_search.parquet')
df_hp

,epoch,type,loss,run_id,modified_embedding_length,batch_size,max_epochs,learning_rate,dropout_fraction
0,1,train,0.092151,1150734315,64,512,30,0.1,0.2
1,1,test,0.081167,1150734315,64,512,30,0.1,0.2
2,2,train,0.078609,1150734315,64,512,30,0.1,0.2
3,2,test,0.079749,1150734315,64,512,30,0.1,0.2
4,3,train,0.096802,1150734315,64,512,30,0.1,0.2
...,...,...,...,...,...,...,...,...,...
415,28,test,0.077820,360739329,2048,512,30,0.1,0.2
416,29,train,0.074052,360739329,2048,512,30,0.1,0.2
417,29,test,0.077724,360739329,2048,512,30,0.1,0.2
418,30,train,0.053180,360739329,2048,512,30,0.1,0.2
